# Descrição dos dados

Nossos dados contém registros sobre a ocorrência de crimes em San Francisco e são divididos em dois arquivos: treino e teste, sendo que o primeiro conjunto possui **877982** exemplos. Trata-se de um dataset ruidoso e que não é fácil, o que aumenta o desafio e exige mais destreza de quem está modelando o problema. Isto contribui em grande medida para o enriquecimento das habilidades de qualquer cientista de dados iniciante.


* **Dates** - timestamp of the crime incident
* **Category** - category of the crime incident (only in train.csv). This is the target variable you are going to predict.
* **Descript** - detailed description of the crime incident (only in train.csv)
* **DayOfWeek** - the day of the week
* **PdDistrict** - name of the Police Department District
* **Resolution** - how the crime incident was resolved (only in train.csv)
* **Address** - the approximate street address of the crime incident 
* **X** - Longitude
* **Y** - Latitude


In [ ]:
!pip install -q imbalanced-learn

import numpy as np
import operator
import string
import pandas as pd
import matplotlib.pyplot as plt

from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import ADASYN

from sklearn import manifold
from sklearn.preprocessing import LabelBinarizer, StandardScaler, Normalizer
from sklearn.model_selection import train_test_split

# Estimadores que vamos testar
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import xgboost as xgb

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import validation_curve
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder

# utilitários para plots

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import folium
from folium import plugins
from folium.plugins import HeatMap
from folium.plugins import FastMarkerCluster
from folium.plugins import MarkerCluster

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

plt.style.use('ggplot')

# Função para plot de alguns gráficos

In [ ]:
sns.set_style("whitegrid")
sns.despine()

def plot_bar(df, title, filename):    
    p = (
        'Set2', 
        'Paired', 
        'colorblind', 
        'husl',
        'Set1', 
        'coolwarm', 
        'RdYlGn', 
        #'spectral'
    )
    color = sns.color_palette(np.random.choice(p), len(df))
    bar   = df.plot(kind='barh',
                    title=title,
                    fontsize=8,
                    figsize=(12,8),
                    stacked=False,
                    width=1,
                    color=color,
    )

    bar.figure.savefig(filename)

    plt.show()

def plot_top_crimes(df, column, title, fname, items=0):
    try:        
        by_col         = df.groupby(column)
        col_freq       = by_col.size()
        col_freq.index = col_freq.index.map(string.capwords)
        col_freq.sort_values(ascending=True, inplace=True)
        plot_bar(col_freq[slice(-1, - items, -1)], title, fname)
    except Exception:
        plot_bar(df, title, fname)

# Importando o dataset

In [ ]:
train_data = pd.read_csv("../input/train.csv", parse_dates =['Dates'])
test_data = pd.read_csv("../input/test.csv", parse_dates =['Dates'])

In [ ]:
print('Shape dos dados de treino:',train_data.shape)
print('Shape dos dados de teste :',test_data.shape)

In [ ]:
train_data.head(6)

# Extraindo features a partir das datas

Datas e horários podem ser bastante informativas. Para o nosso caso, iremos "quebrar" as datas em seu componentes mais elementares, de modo a vermos o que determinados dias ou horários no dizem a respeito da ocorrência de crimes. Um outro efeito disto é que esta prática vai aumentar a quantidade de features disponíveis para o algoritmo. Se tomarmos como exemplo a data **2018-10-28 23:53:00**, nós poderíamos quebrar esta data em:

* **ano:** 2018
* **mês:** 10
* **dia:** 28
* **hora:** 23h
* **minutos:** 53

Na prática, isto nos daria mais 5 colunas em nosso dataframe, aumentando a quantidade de informações.

In [ ]:
# separa as datas em ano, mês, dia, hora, minuto e segundo.
# cada parte da data em uma coluna separada. Isto aumenta a quantidade de features

for x in [train_data, test_data]: 
    x['years'] = x['Dates'].dt.year
    x['months'] = x['Dates'].dt.month
    x['days'] = x['Dates'].dt.day
    x['hours'] = x['Dates'].dt.hour
    x['minutes'] = x['Dates'].dt.minute

# Extraindo features a partir das coordenadas
Vamos compor uma nova coluna no Dataframe apartir do produto entre as features X e Y, que correspondem aos valores de latitude e longitude, respectivamente.

In [ ]:
train_data['XY'] = train_data.X * train_data.Y
test_data['XY'] = test_data.X * test_data.Y

# Visualizações
Pelas visualizações é possível notar que os crimes no distrito de  **SOUTHERN** têm uma frequência mais elevada do que nos outros distritos. Além disso, **entre 01:00 am e 07:00 am os crimes são menos frequentes**. Horários de **maior pico** estão **entre 17:00 e 18:00**.

**Classes desbalanceadas** 

Um problema patente pode ser notado com relação à distribuição das classes ao longo do conjunto de aprendizagem, onde se pode notar que estão **bastante desbalanceadas**. Há classes com menos de 10 exemplos, enquanto há outras com 126182, 174900, etc . Classes desbalanceadas introduzem um viés no modelo, que terá a tendência de favorecer as classes majoritárias.

Eliminar as classes menos representativas podería resultar em um modelo mais estável. O problema disso é que a Kaggle não computaria o nosso score, pois espera que todas as classes estejam presentes no output do nosso modelo.

In [ ]:
plot_top_crimes(train_data, 'Category', 'Por categoria', 'category.png')
# quantidade de crimes associado à cada uma das categorias
print(train_data.Category.value_counts())

In [ ]:
plot_top_crimes(train_data, 'Address', 'Principais localizações de ocorrências',  'location.png', items=50)
print(train_data.Address.value_counts())

In [ ]:
plot_top_crimes(train_data, 'PdDistrict', 'Departamentos com mais atividades',  'police.png')
# quantidade de incidentes associada à cada distrito policial
print(train_data.PdDistrict.value_counts())

In [ ]:
fig, ((axis1,axis2)) = plt.subplots(nrows=1, ncols=2)
fig.set_size_inches(15,4)

sns.countplot(data=train_data, x='days', ax=axis1)
sns.countplot(data=train_data, x='hours', ax=axis2)
plt.show()

Percentual de incidentes por endereço

In [ ]:
addr = train_data['Address'].apply(lambda x: ' '.join(x.split(' ')[-2:]))

year_count=addr.value_counts().reset_index().sort_values(by='index').head(10)
year_count.columns=['addr','Count']
# Create a trace
tag = (np.array(year_count.addr))
sizes = (np.array((year_count['Count'] / year_count['Count'].sum())*100))
plt.figure(figsize=(15,8))

trace = go.Pie(labels=tag, values=sizes)
layout = go.Layout(title='Endereços com mais incidentes')
data = [trace]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename="Inncidentes")


Número de crimes que ocorreram durante cada mês e ao longo dos anos registrados no conjunto de dados. 

In [ ]:
data=[]
for i in range(2003,2015):
    year=train_data[train_data['years']==i]
    year_count=year['months'].value_counts().reset_index().sort_values(by='index')
    year_count.columns=['months','Count']
    trace = go.Scatter(
    x = year_count.months,
    y = year_count.Count,
    name = i)
    data.append(trace)
    

py.iplot(data, filename='basic-line')

Número de crimes por lacalidade (limitamos em 1000 registros)

In [ ]:
m = folium.Map(
    location=[train_data.Y.mean(), train_data.X.mean()],
    tiles='Cartodb Positron',
    zoom_start=13
)

marker_cluster = MarkerCluster(
    name='Locais de crimes em San Francisco',
    overlay=True,
    control=False,
    icon_create_function=None
)
for k in range(1000):
    location = train_data.Y.values[k], train_data.X.values[k]
    marker = folium.Marker(location=location,icon=folium.Icon(color='green'))
    popup = train_data.Address.values[k]
    #popup = train_data.Address.apply(lambda x: ' '.join(x.split(' ')[-2:])).values[k]
    folium.Popup(popup).add_to(marker)
    marker_cluster.add_child(marker)

marker_cluster.add_to(m)

folium.LayerControl().add_to(m)

m.save("cluster.html")

m

**LARCENY/THEFT** é a categoria de crimes com maior número de ocorrências. Onde estas ocorrências mais se concentram?

In [ ]:
new=train_data[train_data['Category']=='LARCENY/THEFT']
M= folium.Map(location=[train_data.Y.mean(), train_data.X.mean() ],tiles= "Stamen Terrain",
                    zoom_start = 13) 

heat_data = [[[row['Y'],row['X']] 
                for index, row in new.head(1000).iterrows()] 
                 for i in range(0,11)]

hm = plugins.HeatMapWithTime(heat_data,auto_play=True,max_opacity=0.8)
hm.add_to(M)

hm.save('heatmap.html')

M

Features altamnte correlacionadas deveriam ser evitadas, já que fornecem pouca informação extra. Neste caso, podemos checar a correlação entre features numéricas.

**Atualização:** há um correlação muito fraca entre as features, como pode ser visto abaixo. 

In [ ]:
# correlações entre as variáveis
train_data.corr().style.format("{:.2}").background_gradient(cmap=plt.get_cmap('tab20c'), axis=1)

# Extraindo features a partir dos endereços

É possível que o fato de o crime ter ocorrido numa rua ou numa avenida possa ter um caráter discriminatório forte. O fato de que avenidas costumam ser mais movimentadas e monitoradas do que muitas ruas pode inviabilizar a prática de determinados crimes. 

Nós podemos extrair esta informação a partir da coluna **'Address'** presente em nosso Dataframe. A string que corresponde ao endereço contém algumas abreviações. Entre estas abreviações nós temos:

* **ST**: Abreviação para Street
* **AV**: Abreviação para Avenue

Nós iremos extrair estas informações, transformá-las em features categóricas e, em seguida, adicionar duas colunas no Dataframe para acomodar estas infos categóricas. Basicamente, estas colunas indicarão se o crime ocorreu numa rua ou numa avenida.

In [ ]:
def street_addr(x):
    street=x.split(' ')
    return (''.join(street[-1]))

train_data['Address_Type'] = train_data['Address'].apply(lambda x:street_addr(x))
test_data['Address_Type'] = test_data['Address'].apply(lambda x:street_addr(x))

for x in [train_data,test_data]:
    x['is_street'] = (x['Address_Type'] == 'ST')
    x['is_avenue'] = (x['Address_Type'] == 'AV')

train_data['is_street'] = train_data['is_street'].apply(lambda x:int(x))
train_data['is_avenue'] = train_data['is_avenue'].apply(lambda x:int(x))

test_data['is_avenue'] = test_data['is_avenue'].apply(lambda x:int(x))
test_data['is_street'] = test_data['is_street'].apply(lambda x:int(x))

Há uma grande quantidade de incidentes cujos endereços associados contém este termo **"Block"** no endereço. Nós podemos criar mais uma feature categórica a partir disto.

In [ ]:
def is_block(x):
    if 'Block' in x:
        return 1
    else:
        return 0

train_data['is_block'] = train_data['Address'].apply(lambda x:is_block(x)) 
test_data['is_block'] = test_data['Address'].apply(lambda x:is_block(x)) 

In [ ]:
train_data.head(20)

# Codificando os rótulos

À princípio, as categorias estão representadas numa forma textual, ou seja, na forma de dados discretos. Contudo, estas informações precisam de uma representação numérica para que nosso modelo possa trabalhar com elas. Nós faremos exatamente isto com os dados da coluna **Category**.

In [ ]:
category = LabelEncoder()
train_data['Category'] = category.fit_transform(train_data.Category)

# Codificando outras features categoricas
Nosso conjunto de dados possui outras features que também são categóricas, como é o caso dos dias da semana nos quais ocorrem os incidentes, bem como o departamento de polícia associado.  Como estas informações estão na forma de texto, nós iremos codificá-las numericamente.

In [ ]:
# codifica outras features categoricas, incluindo-as como novas colunas no dataframe
feature_cols =['DayOfWeek', 'PdDistrict']
train_data = pd.get_dummies(train_data, columns=feature_cols)
test_data = pd.get_dummies(test_data, columns=feature_cols)

# Descartando colunas que não utilizaremos
Nós não precisaremos mais das colunas indicadas abaixo.

In [ ]:
# Nós não precisaremos das colunas abaixo, motivo pelo qual irems descartá-las.
train_data = train_data.drop(['Dates', 'Address', 'Address_Type', 'Resolution'], axis = 1)
train_data = train_data.drop(['Descript'], axis = 1)
test_data = test_data.drop(['Address','Address_Type', 'Dates'], axis = 1)

In [ ]:
train_data.head(5)

In [ ]:
test_data.head(5)

# Modelagem

In [ ]:
feature_cols = [x for x in train_data if x!='Category']
X = train_data[feature_cols]
y = train_data['Category']
X_train, x_test,y_train, y_test = train_test_split(X, y)

# Normalização

Normalizar e padronizar os dados nos previne contra problemas envolvendo as escalas dos números. Além disso, vai permitir que nossos dados tenham as propriedades de uma distribuição normal padrão, com média **0** e desvio padrão **1**. Trata-se de um procedimento, a bem dizer, mandatório em muitas tarefas envolvendo análise de dados.

In [ ]:
del X
del y

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
x_test = scaler.transform(x_test)

normalizer = Normalizer()
X_train = normalizer.fit_transform(X_train)
x_test = normalizer.transform(x_test)


# Tratando classes desbalanceadas

classes desbalanceadas introduzem um viés nos modelos, com forte tendência ao favorecimento da classe majoritária. Abaixo nós tentamos resolver este problema por meio de uma técnica de *oversamplig*, que gera dados sintéticos com bases nos vizinhos mais próximos de determinados segmentos da classe minoritária.

**atualização**

Nenhum dos métodos de oversampling testados abaixo trouxe melhora no desempenho do classificador. Undersampling não seria uma boa alternativa, considerando que perderíamos informação de mais das outras classes para balancear estes rótulos de acordo com as classes minoritárias.

In [ ]:
#ros = RandomOverSampler(random_state=42, sampling_strategy='not majority')
#X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

#sm = SMOTE(random_state=42, k_neighbors=3)
#X_resampled, y_resampled = sm.fit_resample(X_train, y_train)

#ada = ADASYN(random_state=42, n_neighbors=4)
#X_resampled, y_resampled = ada.fit_resample(X_train, y_train)

#from collections import Counter

#print('shape do dataset original %s' % Counter(y_train))
#print('shape do dataset com oversampling %s' % Counter(y_resampled))

#print(sorted(Counter(y_resampled).items()))

## Random Forest

Random Forest é um algorítmo baseado em ensemble bastante popular, que cria vários classificadores, baseados em árvore de decisão, em cima dos dados de treino e combina todas as saídas destes classificadores para obter uma acurácia mais estável.

Outras abordagens testadas aqui, e que não se saíram melhor do que a Random Forest, foram:
1. SVM
2. Multi layer Perceptron
3. Gradient Boosting
4. Regressão Logística

Apesar de nós estarmos usando a métrica acurácia (abaixo), é bom lembrar que ela não é uma medida de desempenho adequada para modelos que lidam com classes desbalanceadas, já que trata todas as classes com igual importância.

**Melhor score alcançado:**

* **accuracy_score**:  0.3242495888626186
* **f1_score_weighted** 0.36493050029624635*

**Melhor score público:** 2.71003

**Score público do líder na competição:** 1.95936

In [ ]:
random_forest = RandomForestClassifier(n_estimators=100, max_depth=23)

In [ ]:
random_forest.fit(X_train, y_train.ravel())
#random_forest.fit(X_resampled, y_resampled)
pred = random_forest.predict(x_test)
print("accuracy_score: ", accuracy_score(pred,y_test))
print("f1_score_weighted", f1_score(pred,y_test, average='weighted'))

# Submissão

In [ ]:
#X_test =test_data.drop(['Id'], axis = 1)
predicted_sub = random_forest.predict_proba(test_data.drop(['Id'], axis = 1))
submission_results = pd.DataFrame(predicted_sub, columns=category.classes_)
submission_results.to_csv('submission.csv', index_label = 'Id')